In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from xgboost import XGBRegressor
from datetime import date, datetime
from sklearn.impute import SimpleImputer
from IPython.core.display import display, HTML
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
%matplotlib inline
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))
print('Libraries imported successfully!!')

Libraries imported successfully!!


In [2]:
db = pd.read_csv('iplav.csv')
db.head()

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
0,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,P Kumar,1,0,0.1,1,0,0,0,222
1,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,1,0,0.2,1,0,0,0,222
2,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.2,2,0,0,0,222
3,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.3,2,0,0,0,222
4,1,18-04-2008,M Chinnaswamy Stadium,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,P Kumar,2,0,0.4,2,0,0,0,222


In [3]:
db.drop(['mid', 'venue', 'batsman', 'bowler', 'striker', 'non-striker'], axis = 1, inplace = True)

In [4]:
db.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total
0,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.1,1,0,222
1,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.2,1,0,222
2,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.2,2,0,222
3,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.3,2,0,222
4,18-04-2008,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.4,2,0,222


In [5]:
db['yyyymmdd'] = db['date'].apply(lambda x : x.split()[0])
db['year'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').year)
db['month'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').month)
db['date'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').day)
db['day_number'] = db['yyyymmdd'].apply(lambda dateString : datetime.strptime(dateString,'%d-%m-%Y').weekday())
db = db.drop(['yyyymmdd'], axis = 1)

In [6]:
db.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number
0,18,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.1,1,0,222,2008,4,4
1,18,Kolkata Knight Riders,Royal Challengers Bangalore,1,0,0.2,1,0,222,2008,4,4
2,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.2,2,0,222,2008,4,4
3,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.3,2,0,222,2008,4,4
4,18,Kolkata Knight Riders,Royal Challengers Bangalore,2,0,0.4,2,0,222,2008,4,4


In [7]:
db.columns

Index(['date', 'bat_team', 'bowl_team', 'runs', 'wickets', 'overs',
       'runs_last_5', 'wickets_last_5', 'total', 'year', 'month',
       'day_number'],
      dtype='object')

In [8]:
for col in ['bat_team', 'bowl_team']:
    t_list = []
    for i in db[col]:
        if i == 'Delhi Daredevils':
            t_list.append('Delhi Capitals')
        elif i == 'Rising Pune Supergiants':
            t_list.append('Rising Pune Supergiant')
        else:
            t_list.append(i)
    db[col] = t_list

In [9]:
df = db[db['year'] >= 2012]
df = df[df['overs'] >= 5.0]

In [10]:
df.head()

,date,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number
29594,4,Chennai Super Kings,Mumbai Indians,35,1,5.1,31,1,112,2012,4,2
29595,4,Chennai Super Kings,Mumbai Indians,36,1,5.2,32,0,112,2012,4,2
29596,4,Chennai Super Kings,Mumbai Indians,37,1,5.3,33,0,112,2012,4,2
29597,4,Chennai Super Kings,Mumbai Indians,38,1,5.4,33,0,112,2012,4,2
29598,4,Chennai Super Kings,Mumbai Indians,38,2,5.5,32,1,112,2012,4,2


In [11]:
dummy = ['bat_team', 'bowl_team']
df_dummy = pd.get_dummies(df[dummy], prefix_sep = '_', drop_first = True)
df = df.drop(dummy, axis = 1)
df = pd.concat([df, df_dummy], axis = 1)

In [12]:
df.head()

,date,runs,wickets,overs,runs_last_5,wickets_last_5,total,year,month,day_number,bat_team_Deccan Chargers,bat_team_Delhi Capitals,bat_team_Gujarat Lions,bat_team_Kings XI Punjab,bat_team_Kolkata Knight Riders,bat_team_Mumbai Indians,bat_team_Pune Warriors,bat_team_Rajasthan Royals,bat_team_Rising Pune Supergiant,bat_team_Royal Challengers Bangalore,bat_team_Sunrisers Hyderabad,bowl_team_Deccan Chargers,bowl_team_Delhi Capitals,bowl_team_Gujarat Lions,bowl_team_Kings XI Punjab,bowl_team_Kolkata Knight Riders,bowl_team_Mumbai Indians,bowl_team_Pune Warriors,bowl_team_Rajasthan Royals,bowl_team_Rising Pune Supergiant,bowl_team_Royal Challengers Bangalore,bowl_team_Sunrisers Hyderabad
29594,4,35,1,5.1,31,1,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
29595,4,36,1,5.2,32,0,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
29596,4,37,1,5.3,33,0,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
29597,4,38,1,5.4,33,0,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
29598,4,38,2,5.5,32,1,112,2012,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [13]:
df.shape

(34685, 32)

In [14]:
df.columns

Index(['date', 'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5',
       'total', 'year', 'month', 'day_number', 'bat_team_Deccan Chargers',
       'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions',
       'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders',
       'bat_team_Mumbai Indians', 'bat_team_Pune Warriors',
       'bat_team_Rajasthan Royals', 'bat_team_Rising Pune Supergiant',
       'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad',
       'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals',
       'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab',
       'bowl_team_Kolkata Knight Riders', 'bowl_team_Mumbai Indians',
       'bowl_team_Pune Warriors', 'bowl_team_Rajasthan Royals',
       'bowl_team_Rising Pune Supergiant',
       'bowl_team_Royal Challengers Bangalore',
       'bowl_team_Sunrisers Hyderabad'],
      dtype='object')

In [15]:
df = df[['runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'total', 'year', 'bat_team_Deccan Chargers', 'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions', 
         'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Pune Warriors', 
         'bat_team_Rajasthan Royals', 'bat_team_Rising Pune Supergiant', 'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad', 
         'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals', 'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders', 
         'bowl_team_Mumbai Indians', 'bowl_team_Pune Warriors', 'bowl_team_Rajasthan Royals', 'bowl_team_Rising Pune Supergiant', 'bowl_team_Royal Challengers Bangalore', 
         'bowl_team_Sunrisers Hyderabad']]

In [16]:
train = df[df['year'] <= 2016]
test = df[df['year'] > 2016]

In [17]:
X_train = train.drop(['year', 'total'], axis = 1)
X_test = test.drop(['year', 'total'], axis = 1)
y_train = train['total']
y_test = test['total']

In [18]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((29344, 27), (5341, 27), (29344,), (5341,))

In [19]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [20]:
y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)
print('Logistic Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_lr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_lr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_lr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_lr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_lr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_lr) / y_test))

Logistic Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6314089443639566
Test r2_score:  0.674575465374619
----------------------------------------------------------------------------------------------------
Train RMSE:  17.417534719935475
Test RMSE:  16.768270127605113
----------------------------------------------------------------------------------------------------
Train MAPE:  0.08377194971941723
Test MAPE:  0.08052532093989041


In [21]:
data_lr = pd.DataFrame()
data_lr['actual'] = y_test
data_lr['predicted'] = y_test_pred_lr
data_lr['lower range'] = y_test_pred_lr - (0.08052532093989041 * y_test_pred_lr)
data_lr['upper range'] = y_test_pred_lr + (0.08052532093989041 * y_test_pred_lr)
t_list = []
for i in data_lr.index:
    if data_lr['actual'][i] > data_lr['lower range'][i]:
        if data_lr['actual'][i] < data_lr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_lr['Bool'] = t_list
data_lr['Bool'].value_counts()/data_lr.shape[0]

True     0.613181
False    0.386819
Name: Bool, dtype: float64

In [22]:
la = Lasso()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
la_rscv = RandomizedSearchCV(estimator = la, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
la_rscv.fit(X_train, y_train)
la_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.1s
[CV] alpha=0.242 .....................................................
[CV] ...................................... alpha=0.242, total=   0.0s
[CV] 

[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.753 .....................................................
[CV] ...................................... alpha=0.753, total=   0.1s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    5.0s finished


{'alpha': 0.279}

In [23]:
y_train_pred_la = la_rscv.predict(X_train)
y_test_pred_la = la_rscv.predict(X_test)
print('Lasso Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_la))
print('Test r2_score: ', r2_score(y_test, y_test_pred_la))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_la)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_la)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_la) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_la) / y_test))

Lasso Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6228137544952084
Test r2_score:  0.6838829107770812
----------------------------------------------------------------------------------------------------
Train RMSE:  17.61944451896617
Test RMSE:  16.526736489748945
----------------------------------------------------------------------------------------------------
Train MAPE:  0.08399641331221015
Test MAPE:  0.07783388032622655


In [33]:
data_la = pd.DataFrame()
data_la['actual'] = y_test
data_la['predicted'] = y_test_pred_la
data_la['lower range'] = y_test_pred_la - (0.07783388032622655 * y_test_pred_la)
data_la['upper range'] = y_test_pred_la + (0.07783388032622655 * y_test_pred_la)
t_list = []
for i in data_la.index:
    if data_la['actual'][i] > data_la['lower range'][i]:
        if data_la['actual'][i] < data_la['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_la['Bool'] = t_list
data_la['Bool'].value_counts()/data_la.shape[0]

True     0.622355
False    0.377645
Name: Bool, dtype: float64

In [25]:
ri = Ridge()
possible_parameter_values = {'alpha' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
ri_rscv = RandomizedSearchCV(estimator = ri, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
ri_rscv.fit(X_train, y_train)
ri_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] ...................................... alpha=0.624, total=   0.0s
[CV] alpha=0.624 .....................................................
[CV] .........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s



[CV] ...................................... alpha=0.242, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] ...................................... alpha=0.242, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] ...................................... alpha=0.242, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] ...................................... alpha=0.242, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] ...................................... alpha=0.242, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] ...................................... alpha=0.242, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] ...................................... alpha=0.242, total=   0.0s
[CV] alpha=0.242 .....................................................
[CV] 

[CV] ...................................... alpha=0.753, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] ...................................... alpha=0.942, total=   0.0s
[CV] alpha=0.942 .....................................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.9s finished


{'alpha': 0.942}

In [26]:
y_train_pred_ri = ri_rscv.predict(X_train)
y_test_pred_ri = ri_rscv.predict(X_test)
print('Ridge Regression:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_ri))
print('Test r2_score: ', r2_score(y_test, y_test_pred_ri))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_ri)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_ri)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_ri) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_ri) / y_test))

Ridge Regression:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.6314089296026026
Test r2_score:  0.6746191962334509
----------------------------------------------------------------------------------------------------
Train RMSE:  17.417535068704645
Test RMSE:  16.76714342169865
----------------------------------------------------------------------------------------------------
Train MAPE:  0.083771275297166
Test MAPE:  0.08051806107511772


In [34]:
data_ri = pd.DataFrame()
data_ri['actual'] = y_test
data_ri['predicted'] = y_test_pred_ri
data_ri['lower range'] = y_test_pred_ri - (0.08051806107511772 * y_test_pred_ri)
data_ri['upper range'] = y_test_pred_ri + (0.08051806107511772 * y_test_pred_ri)
t_list = []
for i in data_ri.index:
    if data_ri['actual'][i] > data_ri['lower range'][i]:
        if data_ri['actual'][i] < data_ri['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_ri['Bool'] = t_list
data_ri['Bool'].value_counts()/data_ri.shape[0]

True     0.613181
False    0.386819
Name: Bool, dtype: float64

In [30]:
dtr = DecisionTreeRegressor()
possible_parameter_values = {'criterion' : ['mse', 'friedman_mse', 'mae'],
                             'splitter' : ['best', 'random'],
                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)],
                             'min_samples_split' : [int(x) for x in np.arange(start = 5, stop = 101, step = 1)],
                             'min_samples_leaf' : [int(x) for x in np.arange(start = 1, stop = 51, step = 1)]}
dtr_rscv = RandomizedSearchCV(estimator = dtr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
dtr_rscv.fit(X_train, y_train)
dtr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.1s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.0s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.0s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.0s
[CV] splitter=random, min_samples_split=44, min_sampl

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.0s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.0s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.0s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.0s
[CV] splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse 
[CV]  splitter=random, min_samples_split=44, min_samples_leaf=48, max_depth=55, criterion=friedman_mse, total=   0.0

[CV]  splitter=random, min_samples_split=84, min_samples_leaf=49, max_depth=50, criterion=friedman_mse, total=   0.1s
[CV] splitter=best, min_samples_split=39, min_samples_leaf=19, max_depth=90, criterion=friedman_mse 
[CV]  splitter=best, min_samples_split=39, min_samples_leaf=19, max_depth=90, criterion=friedman_mse, total=   0.1s
[CV] splitter=best, min_samples_split=39, min_samples_leaf=19, max_depth=90, criterion=friedman_mse 
[CV]  splitter=best, min_samples_split=39, min_samples_leaf=19, max_depth=90, criterion=friedman_mse, total=   0.1s
[CV] splitter=best, min_samples_split=39, min_samples_leaf=19, max_depth=90, criterion=friedman_mse 
[CV]  splitter=best, min_samples_split=39, min_samples_leaf=19, max_depth=90, criterion=friedman_mse, total=   0.1s
[CV] splitter=best, min_samples_split=39, min_samples_leaf=19, max_depth=90, criterion=friedman_mse 
[CV]  splitter=best, min_samples_split=39, min_samples_leaf=19, max_depth=90, criterion=friedman_mse, total=   0.1s
[CV] splitter=

[CV]  splitter=random, min_samples_split=69, min_samples_leaf=11, max_depth=70, criterion=mae, total=  23.2s
[CV] splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse 
[CV]  splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse, total=   0.0s
[CV] splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse 
[CV]  splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse, total=   0.0s
[CV] splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse 
[CV]  splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse, total=   0.0s
[CV] splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse 
[CV]  splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse, total=   0.0s
[CV] splitter=best, min_samples_split=28, min_samples_leaf=10, max_depth=5, criterion=mse 
[CV]  splitt

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 10.2min finished


{'splitter': 'best',
 'min_samples_split': 28,
 'min_samples_leaf': 10,
 'max_depth': 5,
 'criterion': 'mse'}

In [52]:
y_train_pred_dtr = dtr_rscv.predict(X_train)
y_test_pred_dtr = dtr_rscv.predict(X_test)
print('Decision Tree Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_dtr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_dtr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_dtr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_dtr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_dtr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_dtr) / y_test))

Decision Tree Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5403038150780984
Test r2_score:  0.5179605329607417
----------------------------------------------------------------------------------------------------
Train RMSE:  19.451349556358952
Test RMSE:  20.408186488954648
----------------------------------------------------------------------------------------------------
Train MAPE:  0.09675354259904664
Test MAPE:  0.09856665840867859


In [53]:
data_dtr = pd.DataFrame()
data_dtr['actual'] = y_test
data_dtr['predicted'] = y_test_pred_dtr
data_dtr['lower range'] = y_test_pred_dtr - (0.09856665840867859 * y_test_pred_dtr)
data_dtr['upper range'] = y_test_pred_dtr + (0.09856665840867859 * y_test_pred_dtr)
t_list = []
for i in data_dtr.index:
    if data_dtr['actual'][i] > data_dtr['lower range'][i]:
        if data_dtr['actual'][i] < data_dtr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_dtr['Bool'] = t_list
data_dtr['Bool'].value_counts()/data_dtr.shape[0]

True     0.630594
False    0.369406
Name: Bool, dtype: float64

In [37]:
rfr = RandomForestRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)],
                             'min_samples_split' : [int(x) for x in np.arange(start = 5, stop = 101, step = 1)],
                             'min_samples_leaf' : [int(x) for x in np.arange(start = 1, stop = 51, step = 1)]}
rfr_rscv = RandomizedSearchCV(estimator = rfr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
rfr_rscv.fit(X_train, y_train)
rfr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=   7.9s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.8s remaining:    0.0s


[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=   7.9s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=   8.2s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=   8.3s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=   8.1s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=   8.2s
[CV] n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55 
[CV]  n_estimators=200, min_samples_split=68, min_samples_leaf=18, max_depth=55, total=   8.4s
[CV] n_estimators=200, min_sam

[CV]  n_estimators=500, min_samples_split=65, min_samples_leaf=32, max_depth=45, total=  19.4s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   1.7s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   1.8s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   1.8s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   1.9s
[CV] n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10 
[CV]  n_estimators=50, min_samples_split=49, min_samples_leaf=16, max_depth=10, total=   1.7s
[CV] n_estimators=50, min_samples_split=

[CV]  n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20, total=   9.2s
[CV] n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20 
[CV]  n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20, total=   9.2s
[CV] n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20 
[CV]  n_estimators=250, min_samples_split=85, min_samples_leaf=37, max_depth=20, total=   9.0s


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 29.8min finished


{'n_estimators': 800,
 'min_samples_split': 89,
 'min_samples_leaf': 50,
 'max_depth': 15}

In [54]:
y_train_pred_rfr = rfr_rscv.predict(X_train)
y_test_pred_rfr = rfr_rscv.predict(X_test)
print('Random Forest Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_rfr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_rfr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_rfr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_rfr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_rfr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_rfr) / y_test))

Random Forest Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.7391073551624214
Test r2_score:  0.6600300398721091
----------------------------------------------------------------------------------------------------
Train RMSE:  14.653629494626045
Test RMSE:  17.138917471265774
----------------------------------------------------------------------------------------------------
Train MAPE:  0.07003273976873108
Test MAPE:  0.080638698151815


In [55]:
data_rfr = pd.DataFrame()
data_rfr['actual'] = y_test
data_rfr['predicted'] = y_test_pred_rfr
data_rfr['lower range'] = y_test_pred_rfr - (0.080638698151815 * y_test_pred_rfr)
data_rfr['upper range'] = y_test_pred_rfr + (0.080638698151815 * y_test_pred_rfr)
t_list = []
for i in data_rfr.index:
    if data_rfr['actual'][i] > data_rfr['lower range'][i]:
        if data_rfr['actual'][i] < data_rfr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_rfr['Bool'] = t_list
data_rfr['Bool'].value_counts()/data_rfr.shape[0]

True     0.621045
False    0.378955
Name: Bool, dtype: float64

In [40]:
abr = AdaBoostRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)],
                             'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
abr_rscv = RandomizedSearchCV(estimator = abr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
abr_rscv.fit(X_train, y_train)
abr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] n_estimators=200, learning_rate=0.544 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.6s
[CV] n_estimators=200, learning_rate=0.544 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.2s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.2s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.2s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.5s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   3.1s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.2s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] ............ n_estimators=200, learning_rate=0.544, total=   1.8s
[CV] n_estimators=200, learning_rate=0.544 ...........................
[CV] .

[CV] ............ n_estimators=250, learning_rate=0.229, total=   3.3s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   0.8s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   0.9s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   1.1s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   1.1s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   1.9s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] ............ n_estimators=650, learning_rate=0.713, total=   1.8s
[CV] n_estimators=650, learning_rate=0.713 ...........................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  6.5min finished


{'n_estimators': 350, 'learning_rate': 0.021}

In [56]:
y_train_pred_abr = abr_rscv.predict(X_train)
y_test_pred_abr = abr_rscv.predict(X_test)
print('Ada Boost Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_abr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_abr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_abr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_abr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_abr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_abr) / y_test))

Ada Boost Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.5093056983147664
Test r2_score:  0.48907891119522984
----------------------------------------------------------------------------------------------------
Train RMSE:  20.09647078739312
Test RMSE:  21.01067620367742
----------------------------------------------------------------------------------------------------
Train MAPE:  0.10141690985856325
Test MAPE:  0.10076419335734588


In [57]:
data_abr = pd.DataFrame()
data_abr['actual'] = y_test
data_abr['predicted'] = y_test_pred_abr
data_abr['lower range'] = y_test_pred_abr - (0.10076419335734588 * y_test_pred_abr)
data_abr['upper range'] = y_test_pred_abr + (0.10076419335734588 * y_test_pred_abr)
t_list = []
for i in data_abr.index:
    if data_abr['actual'][i] > data_abr['lower range'][i]:
        if data_abr['actual'][i] < data_abr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_abr['Bool'] = t_list
data_abr['Bool'].value_counts()/data_abr.shape[0]

True     0.603071
False    0.396929
Name: Bool, dtype: float64

In [45]:
knr = KNeighborsRegressor()
possible_parameter_values = {'n_neighbors' : [int(x) for x in np.arange(start = 1, stop = 11, step = 1)],
                             'weights' : ['uniform', 'distance']}
knr_rscv = RandomizedSearchCV(estimator = knr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, random_state = 17, 
                              n_jobs = 1)
knr_rscv.fit(X_train, y_train)
knr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] weights=uniform, n_neighbors=8 ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................... weights=uniform, n_neighbors=8, total=   0.3s
[CV] weights=uniform, n_neighbors=8 ..................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] ................... weights=uniform, n_neighbors=8, total=   0.3s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.3s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.3s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.3s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.3s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.3s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] ................... weights=uniform, n_neighbors=8, total=   0.3s
[CV] weights=uniform, n_neighbors=8 ..................................
[CV] .

[CV] ................... weights=uniform, n_neighbors=5, total=   0.3s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.3s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.3s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.3s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.3s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.3s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] ................... weights=uniform, n_neighbors=7, total=   0.3s
[CV] weights=uniform, n_neighbors=7 ..................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   26.8s finished


{'weights': 'uniform', 'n_neighbors': 8}

In [58]:
y_train_pred_knr = knr_rscv.predict(X_train)
y_test_pred_knr = knr_rscv.predict(X_test)
print('K Neighbors Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_knr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_knr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_knr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_knr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_knr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_knr) / y_test))

K Neighbors Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.8109540090753671
Test r2_score:  0.5649839016463882
----------------------------------------------------------------------------------------------------
Train RMSE:  12.473778407189613
Test RMSE:  19.387230770028765
----------------------------------------------------------------------------------------------------
Train MAPE:  0.056638190226127684
Test MAPE:  0.09076316522754599


In [59]:
data_knr = pd.DataFrame()
data_knr['actual'] = y_test
data_knr['predicted'] = y_test_pred_knr
data_knr['lower range'] = y_test_pred_knr - (0.09076316522754599 * y_test_pred_knr)
data_knr['upper range'] = y_test_pred_knr + (0.09076316522754599 * y_test_pred_knr)
t_list = []
for i in data_knr.index:
    if data_knr['actual'][i] > data_knr['lower range'][i]:
        if data_knr['actual'][i] < data_knr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_knr['Bool'] = t_list
data_knr['Bool'].value_counts()/data_knr.shape[0]

True     0.633028
False    0.366972
Name: Bool, dtype: float64

In [49]:
xgbr = XGBRegressor()
possible_parameter_values = {'n_estimators' : [int(x) for x in np.arange(start = 50, stop = 1001, step = 50)], 
                             'use_label_encoder' : [True, False], 
                             'max_depth' : [int(x) for x in np.arange(start = 5, stop = 101, step = 5)], 
                             'learning_rate' : [float(x)/1000 for x in np.arange(start = 1, stop = 1001, step = 1)]}
xgbr_rscv = RandomizedSearchCV(estimator = xgbr, param_distributions = possible_parameter_values, cv = 10, scoring = 'neg_mean_squared_error', verbose = 2, 
                               random_state = 17, n_jobs = 1)
xgbr_rscv.fit(X_train, y_train)
xgbr_rscv.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   5.0s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.9s remaining:    0.0s


[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   3.9s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   3.7s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   4.1s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   4.8s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   4.8s
[CV] use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737 
[CV]  use_label_encoder=False, n_estimators=50, max_depth=35, learning_rate=0.737, total=   4.5s
[CV] use

[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=   4.7s
[CV] use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971 
[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=   4.9s
[CV] use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971 
[CV]  use_label_encoder=False, n_estimators=750, max_depth=15, learning_rate=0.971, total=   4.7s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=   5.5s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=   5.7s
[CV] use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646 
[CV]  use_label_encoder=True, n_estimators=400, max_depth=50, learning_rate=0.646, total=   5.5s
[CV

[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  46.7s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  45.2s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  43.1s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  42.6s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  43.9s
[CV] use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1 
[CV]  use_label_encoder=True, n_estimators=750, max_depth=75, learning_rate=0.1, total=  47.2s
[CV] use_label_encoder=True, n

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 17.4min finished


{'use_label_encoder': True,
 'n_estimators': 50,
 'max_depth': 15,
 'learning_rate': 0.158}

In [60]:
y_train_pred_xgbr = xgbr_rscv.predict(X_train)
y_test_pred_xgbr = xgbr_rscv.predict(X_test)
print('XGBoost Regressor:')
print('-' * 100)
print('Train r2_score: ', r2_score(y_train, y_train_pred_xgbr))
print('Test r2_score: ', r2_score(y_test, y_test_pred_xgbr))
print('-' * 100)
print('Train RMSE: ', np.sqrt(mean_squared_error(y_train, y_train_pred_xgbr)))
print('Test RMSE: ', np.sqrt(mean_squared_error(y_test, y_test_pred_xgbr)))
print('-' * 100)
print('Train MAPE: ', np.mean(mean_absolute_error(y_train, y_train_pred_xgbr) / y_train))
print('Test MAPE: ', np.mean(mean_absolute_error(y_test, y_test_pred_xgbr) / y_test))

XGBoost Regressor:
----------------------------------------------------------------------------------------------------
Train r2_score:  0.9953679630630898
Test r2_score:  0.5576490541651531
----------------------------------------------------------------------------------------------------
Train RMSE:  1.9525416009205305
Test RMSE:  19.549992516316976
----------------------------------------------------------------------------------------------------
Train MAPE:  0.0076132801897291985
Test MAPE:  0.0915860626361764


In [61]:
data_xgbr = pd.DataFrame()
data_xgbr['actual'] = y_test
data_xgbr['predicted'] = y_test_pred_xgbr
data_xgbr['lower range'] = y_test_pred_xgbr - (0.0915860626361764 * y_test_pred_xgbr)
data_xgbr['upper range'] = y_test_pred_xgbr + (0.0915860626361764 * y_test_pred_xgbr)
t_list = []
for i in data_xgbr.index:
    if data_xgbr['actual'][i] > data_xgbr['lower range'][i]:
        if data_xgbr['actual'][i] < data_xgbr['upper range'][i]:
            t_list.append('True')
        else:
            t_list.append('False')
    else:
        t_list.append('False')
data_xgbr['Bool'] = t_list
data_xgbr['Bool'].value_counts()/data_xgbr.shape[0]

True     0.619547
False    0.380453
Name: Bool, dtype: float64

In [ ]:
'runs', 'wickets', 'overs', 'runs_last_5', 'wickets_last_5', 'total', 'year', 'bat_team_Deccan Chargers', 'bat_team_Delhi Capitals', 'bat_team_Gujarat Lions', 
         'bat_team_Kings XI Punjab', 'bat_team_Kolkata Knight Riders', 'bat_team_Mumbai Indians', 'bat_team_Pune Warriors', 
         'bat_team_Rajasthan Royals', 'bat_team_Rising Pune Supergiant', 'bat_team_Royal Challengers Bangalore', 'bat_team_Sunrisers Hyderabad', 
         'bowl_team_Deccan Chargers', 'bowl_team_Delhi Capitals', 'bowl_team_Gujarat Lions', 'bowl_team_Kings XI Punjab', 'bowl_team_Kolkata Knight Riders', 
         'bowl_team_Mumbai Indians', 'bowl_team_Pune Warriors', 'bowl_team_Rajasthan Royals', 'bowl_team_Rising Pune Supergiant', 'bowl_team_Royal Challengers Bangalore', 
         'bowl_team_Sunrisers Hyderabad'

In [20]:
knr_rscv = KNeighborsRegressor(weights = 'uniform', n_neighbors = 8)
knr_rscv.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=8)

In [36]:
arr = np.array([171, 5, 19, 45, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [37]:
arr = arr.reshape(1, -1)

In [38]:
s = knr_rscv.predict(arr)[0]
s

187.125

In [39]:
s - (0.09076316522754599 * s)

170.14094270679544

In [40]:
s + (0.09076316522754599 * s)

204.10905729320456